In [1]:
import os
import io
import cv2
import csv

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import transforms, models, datasets

import matplotlib.pyplot as plt
from PIL import Image
from torch import optim
import glob, numpy as np, pandas as pd

from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
model = models.resnet50(pretrained=True)
 
train_data_dir = r'C:\Users\fano\Desktop\case\techosmotr\techosmotr\train'
test_data_dir = r'C:\Users\fano\Desktop\case\techosmotr\techosmotr\test'

C:\Users\fano\anaconda3\envs\wobblers\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\fano\anaconda3\envs\wobblers\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
class Cars(Dataset):
    def __init__(self, folder):

        photoshop=[os.path.normpath(i) for i in glob(folder+'/photoshop/*.jpeg')]
        screen=[os.path.normpath(i) for i in glob(folder+'/screen/*.jpeg')]
        screen_photoshop=[os.path.normpath(i) for i in glob(folder+'/screen_photoshop/*.jpeg')]
        correct=[os.path.normpath(i) for i in glob(folder+'/correct/*.jpeg')]
        not_station=[os.path.normpath(i) for i in glob(folder+'/not_on_station/*.jpeg')]
        
        self.fpaths=photoshop+screen+screen_photoshop+correct+not_station
        self.aug_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.Normalize(mean=[0.45437595,0.45985883,0.46735263], std=[0.2684269,0.26880786,0.2686823])
        ])
        from random import shuffle, seed; seed(10);
        shuffle(self.fpaths)
        self.targets=[]
        #print(self.fpaths[0])
        for file in self.fpaths:
            tmp = [0,0,0,0,0] 
            if file.startswith(r'C:\Users\fano\Desktop\case\techosmotr\techosmotr\train\photoshop'):
                tmp[0] = 1
            elif file.startswith(r'C:\Users\fano\Desktop\case\techosmotr\techosmotr\train\screen'):
                tmp[1] = 1
            elif file.startswith(r'C:\Users\fano\Desktop\case\techosmotr\techosmotr\train\screen_photoshop'):
                tmp[2] = 1
            elif file.startswith(r'C:\Users\fano\Desktop\case\techosmotr\techosmotr\train\correct'):
                tmp[3] = 1
            elif file.startswith(r'C:\Users\fano\Desktop\case\techosmotr\techosmotr\train\not_on_station'):
                tmp[4] = 1
            self.targets.append(tmp)

    def __len__(self):
        return len(self.fpaths)

    def __getitem__(self, ix):
        f = self.fpaths[ix]
        target = self.targets[ix]
        im = cv2.imread(f)[:,:,::-1]
        im = cv2.resize(im, (224,224))
        im = torch.tensor(im/255)
        im = im.permute(2,0,1)
        im = self.aug_transforms(im)
        return im.float().to(device), torch.tensor([target]).float().to(device).squeeze(0)


In [4]:
cars = Cars(train_data_dir)

In [5]:
len(cars)

6585

In [6]:
cars[0]

(tensor([[[ 0.3380,  0.4549,  0.4695,  ...,  0.0750, -1.1814, -0.0273],
          [ 0.3672,  0.4549,  0.4549,  ..., -0.0126, -0.6701, -0.0126],
          [ 0.4110,  0.5279,  0.3526,  ..., -0.0273, -0.0711,  0.0458],
          ...,
          [-0.4363,  1.3168,  1.1123,  ..., -0.6847, -0.6262, -0.6262],
          [-0.8600,  1.3606,  1.0392,  ..., -0.6847, -0.6262, -0.6262],
          [-1.0499,  1.1561,  0.9370,  ..., -0.6409, -0.6555, -0.6262]],
 
         [[ 0.2733,  0.3025,  0.3171,  ...,  0.2442, -0.9521,  0.1566],
          [ 0.3025,  0.3025,  0.3317,  ...,  0.1858, -0.4853,  0.1566],
          [ 0.3025,  0.3900,  0.2587,  ...,  0.1712,  0.0983,  0.2150],
          ...,
          [-0.3102,  1.2362,  1.0465,  ..., -0.6749, -0.6020, -0.6166],
          [-0.6603,  1.2945,  1.0028,  ..., -0.6603, -0.6020, -0.6312],
          [-0.7625,  1.1049,  0.9298,  ..., -0.6166, -0.6312, -0.6312]],
 
         [[ 0.1726,  0.1434,  0.1580,  ...,  0.3477, -0.8637,  0.2018],
          [ 0.2018,  0.1434,

In [7]:
def get_model2():
    for param in model.parameters():
        param.requires_grad = False
    counter = 0
    for param in model.parameters():
        counter += 1
        if counter > 150:
            param.requires_grad = True
            
#nn.Linear(2048,512),
    model.fc = nn.Linear(2048,5)

    loss = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(model.parameters(), lr=1e-3)
    return model.to(device), loss, optim


model, criterion, optimizer = get_model2()

In [8]:
from torchsummary import summary

summary(model, (3, 224, 224))
#model.eval()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [9]:
def train_batch(x, y, model, loss, optim):
    model.train(),
    prediction = model(x)
    loss_value = loss(prediction, y)
    loss_value.backward()
    optim.step()
    optim.zero_grad()
    return loss_value.item()


def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    is_correct = torch.argmax(prediction) == torch.argmax(y)
    return is_correct.cpu().numpy().tolist()


In [10]:
cars = Cars(train_data_dir)
validation_split = 0.05
seed = 10

indxs = list(range(len(cars)))
split = int(np.floor(validation_split * len(cars)))
np.random.seed(seed)
np.random.shuffle(indxs)
trn_ind, val_ind = indxs[split:], indxs[:split]

trn_s = SubsetRandomSampler(trn_ind)
val_s = SubsetRandomSampler(val_ind)

trn_dl = DataLoader(cars,batch_size=64,sampler=trn_s)
vld_dl = DataLoader(cars,batch_size=1,sampler=val_s)

train_accuracies, train_losses = [], []
val_accuracies = []
best_val_epoch_accuracy = 0

In [11]:
from tqdm import tqdm

In [12]:
for epoch in range(20):
    print(f" epoch {epoch + 1}/20")
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies = []
    for ix, batch in tqdm(enumerate(iter(trn_dl))):
        x, y = batch

        batch_loss = train_batch(x, y, model, criterion, optimizer)
        train_epoch_losses.append(batch_loss)
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        for photo, label in zip(x, y):
            is_correct = accuracy(torch.unsqueeze(photo,0), torch.unsqueeze(label,0), model)
            train_epoch_accuracies.append(is_correct)

    train_epoch_loss = np.array(train_epoch_losses).mean()
    train_epoch_accuracy = np.mean(train_epoch_accuracies)
    for ix, batch in enumerate(iter(vld_dl)):
        x, y = batch
        val_acc = accuracy(x, y, model)
        val_epoch_accuracies.append(val_acc)
    val_epoch_accuracy = np.mean(val_epoch_accuracies)
    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)
    print("loss at {} epoch is ".format(epoch + 1) + str(train_epoch_loss))
    print("accuracy at {} epoch is ".format(epoch + 1) + str(train_epoch_accuracy * 100) + "%")
    if val_epoch_accuracy > best_val_epoch_accuracy:
        best_val_epoch_accuracy = val_epoch_accuracy
        torch.save(model, r"C:\Users\fano\Desktop\case\best.pt")
    print("validation accuracy at {} epoch is ".format(epoch + 1) + str(val_epoch_accuracy * 100) + "%")


 epoch 1/20


98it [01:38,  1.00s/it]


loss at 1 epoch is 0.3270181891869526
accuracy at 1 epoch is 95.5242966751918%
validation accuracy at 1 epoch is 91.7933130699088%
 epoch 2/20


98it [01:39,  1.02s/it]


loss at 2 epoch is 0.15637852669674523
accuracy at 2 epoch is 97.41048593350384%
validation accuracy at 2 epoch is 91.1854103343465%
 epoch 3/20


...

loss at 18 epoch is 0.01604846082699048
accuracy at 18 epoch is 99.82416879795396%
validation accuracy at 18 epoch is 91.1854103343465%
 epoch 19/20


98it [01:42,  1.04s/it]


loss at 19 epoch is 0.010566811600274749
accuracy at 19 epoch is 99.95204603580564%
validation accuracy at 19 epoch is 92.40121580547113%
 epoch 20/20


98it [01:39,  1.02s/it]


loss at 20 epoch is 0.006379285836039938
accuracy at 20 epoch is 99.95204603580564%
validation accuracy at 20 epoch is 91.48936170212765%


In [13]:
PATH=r"C:\Users\fano\Desktop\case\last.pt"
torch.save(model,PATH)


In [14]:
model = torch.load(PATH)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
#test is ABSOLUTE_PATH_TO_THE_DIRECTORY_WITH_TEST_IMAGES
tests = [os.path.normpath(i) for i in glob(test_data_dir+'/*.jpeg')]


In [16]:
len(tests)

777

In [17]:
def image_loader(image_name):    
    image = cv2.imread(image_name)[:,:,::-1]
    image = cv2.resize(image, (224,224))
    image = torch.tensor(image/255)
    image = image.permute(2,0,1)
    normalize = transforms.Normalize(mean=[0.45437595,0.45985883,0.46735263], std=[0.2684269,0.26880786,0.2686823])
    #normalize=transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    image = normalize(image)
    image = image.unsqueeze(0)  
    return image.float().to(device)


results = []

# 0 - photoshop 1 - screen 2 - screen_photoshop 3 - correct 4 - not_station

    

with open(r"C:\Users\fano\Desktop\case\test_lables.csv",'w',encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["file_index","class"])
    with torch.no_grad():    
        for test in tqdm(tests):
            image = image_loader(test)
            name = test[54:-5]
            value = 0 if torch.argmax(model(image)).item() == 3 else 1
            result = [f"{name}",f"{value}"]
            
            writer.writerow(result)



100%|████████████████████████████████████████████████████████████████████████████████| 777/777 [00:15<00:00, 49.37it/s]
